In [22]:
#!pip install packagename
# importing modules
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import os
from os import chdir as cd
import time
import fiona

In [23]:
US_places =  gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\compiled.shp')
US_places = US_places.to_crs('EPSG:9311')

# dropping columns that will not be used in the anaysis
places_US = US_places.drop(['PLACEFP', 'PLACENS', 'NAME', 'LSAD', 'CLASSFP',
                            'PCICBSA', 'PCINECTA', 'MTFCC', 'FUNCSTAT', 
                            'AWATER', 'INTPTLAT', 'INTPTLON', ], axis =1)

In [24]:
US_places['CLASSFP'].unique()

array(['C1', 'U1', 'U2', 'M2', 'C8', 'C5', 'C9', 'C6', 'C2', 'C7'],
      dtype=object)

In [25]:
# import shapefiles and merge by GEOID
# ========================================
# RUN ONCE TO CRETAE THE COMPILED FILE
# from pathlib import Path
# # define the file location
# folder = Path(r"D:\Work\Box Sync\Trends_all states\Census Tract HUs\\")
# # reading the zip file
# shapefiles = folder.glob(r"Shapefiles\tl_2020_*_tract.zip")
# gdf_CTs = pd.concat([gpd.read_file(shp) for shp in shapefiles]).pipe(gpd.GeoDataFrame)
# gdf_CTs = gdf_CTs.set_crs("EPSG:4269")
# gdf_CTs.to_file(folder / 'compiled_CTs.shp')

US_CTs = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Census Tract HUs\compiled_CTs.shp')
US_CTs.head()
US_CTs = US_CTs.to_crs('EPSG:9311')

In [26]:
US_CTs.shape

(85528, 13)

In [5]:
df_HousingUnits = pd.read_csv(r'D:\Work\Box Sync\Trends_all states\Census Tract HUs\ACSDT5Y2020.B25001-Data.csv')
df_HousingUnits.head()
df_HousingUnits.columns
# extracting only necessary columns
df_HUs = df_HousingUnits.iloc[1:,:3].reset_index()

In [6]:
# checking size of the census tract geographic data and the household units at census tract level from ACS 
US_CTs.shape, df_HUs.shape

((85528, 13), (85395, 4))

In [7]:
df_HUs['GEOID'] = df_HUs['GEO_ID'].str[9:]

In [8]:
# merging Census tract housing unit data with geography 
# outer keeps all values for both dataframes, indicator adds a columns _merge with indicator
df_censusTracts = US_CTs.merge(df_HUs, how='outer', on  = 'GEOID',  indicator = True)

# # unmatched rows
# df_censusTracts[df_censusTracts['_merge'] == 'left_only']

In [9]:
df_censusTracts[df_censusTracts['_merge'] == 'left_only'].shape

(133, 18)

In [10]:
# unmatched values are out of mainland USA
df_censusTracts[df_censusTracts['_merge'] == 'left_only']['STATEFP'].unique()

array(['60', '66', '69', '78'], dtype=object)

In [11]:
# converting census tract area to sqmile, TigerLine shapefile unit is square meter
df_censusTracts['CT_area_sqmi'] = df_censusTracts['ALAND'] * 0.386102/ 1000000
# changing total housing unit column to float from str
df_censusTracts['B25001_001E'] = df_censusTracts['B25001_001E'].astype(float)
# changing column name for better understanding
df_censusTracts.rename(columns = {'B25001_001E':'HousingUnits'}, inplace = True)
# Housing density in each census tract as HUs/square miles
# keeping the values in sqmile to compare with this report
# report link: https://bjs.ojp.gov/library/publications/classification-urban-suburban-and-rural-areas-national-crime-victimization
df_censusTracts['HU_density'] = df_censusTracts['HousingUnits'] / df_censusTracts['CT_area_sqmi']

# dropping columns that will not be used in the anaysis
df_censusTracts = df_censusTracts.drop(['COUNTYFP', 'TRACTCE', 'NAME_x', 'NAMELSAD', 'MTFCC', 
                                        'FUNCSTAT', 'AWATER', 'INTPTLAT', 'INTPTLON',
                                        'index', 'GEO_ID', '_merge'], axis = 1)

In [12]:
state_ids = df_censusTracts['STATEFP'].sort_values().unique()

In [13]:
# places_US['STATEFP'].nunique(), df_censusTracts['STATEFP'].nunique()

In [14]:
df_overlayed = pd.DataFrame()
for id_value in state_ids:
    df_housingunits = df_censusTracts[df_censusTracts['STATEFP'] == id_value]
    df_places = places_US[places_US['STATEFP'] == id_value]
    tract_place_overlayed = gpd.overlay(df_housingunits, df_places, how='intersection', keep_geom_type=False)
    
    df_overlayed = pd.concat([df_overlayed, tract_place_overlayed], axis=0, ignore_index=True)

In [15]:
# what percent of census tract area is in that place
# calculating percent area of census tract that intersected with places and rounding to 2 decimal places
df_overlayed['intersect_area'] = df_overlayed.area
df_overlayed['area%'] = (df_overlayed['intersect_area'] / df_overlayed['ALAND_1']).round(2)
# replacing inf values with zero
df_overlayed['area%'].replace([np.inf, -np.inf], 0, inplace=True) 
# setting %area above 1 to 1 since more than 100% area cannot be in a place
df_overlayed['area%'] = df_overlayed['area%'].apply(lambda x: x if x <= 1 else 1)
# calculating no of housing units in the intersected area
df_overlayed['HU_inArea'] = df_overlayed['HousingUnits'] * df_overlayed['area%']
# housing units in the area weighted by housing units
df_overlayed['HU_weighted_sum'] = df_overlayed['HU_inArea'] * df_overlayed['HU_density'] 

In [16]:
# df_overlayed.columns

In [17]:
df_weighted_density = df_overlayed.groupby(['GEOID_2']).agg({'NAMELSAD': 'first',
                                                             'ALAND_2' : 'first',
                                                             'intersect_area': 'sum',
                                                             'HU_density': 'sum',
                                                             'HU_inArea':'sum',
                                                             'HU_weighted_sum':'sum',}).reset_index()
df_weighted_density['HU_weighted_sum'].isna().sum()

0

In [18]:
# df_weighted_density[df_weighted_density['HU_weighted_sum'] == 0]

In [19]:
# taking weighted average for each place after applying groupby
df_weighted_density['weighted_HU_density'] = df_weighted_density['HU_weighted_sum']/ df_weighted_density['HU_inArea']
# replacing NaN values
df_weighted_density['weighted_HU_density'] = df_weighted_density['weighted_HU_density'].replace(np.nan, -99)

In [20]:
df_weighted_density.to_csv('D:\Work\Box Sync\Trends_all states\Output from Analysis\weighted_housingUnits_for_places.csv')

In [21]:
# df_weighted_density.sort_values('weighted_HU_density').tail(30)